# Bütün Modeller İçin Kullanılacak Ortak Test Verisetinin Hazırlanması

Burada bütün modellerde ayrılan, Train ve Test'in test kısmını alarak Majority Vote'da test edilmesini sağlayacağız.
- Verisetini import ettikten sonra kategorikleri Numerik yaparak Bütün modeller için hazır hale getiriyoruz. Tabii, Derin öğrenme ve Olasılık Bazlı modeller kendi içerisinde özel birkaç ayar daha yapıyorlar, onlarında fonksiyonları bulunmakta.

In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.models import model_from_json
from sklearn.preprocessing import MinMaxScaler

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Verisetinin import edilmesi, duplicate değerler'in düşürülmesi
test = pd.read_excel("../Datasets/X_test.xlsx")
test = test.drop_duplicates()
target = test.Churn
test.drop("Churn", axis=1, inplace=True)
features = list(test.columns)

In [3]:
#Kategorik verilerin numerik hale getirilmesi
le = preprocessing.LabelEncoder()

objects = ["SON5ISTIHBARAT_SONUC","SON5POLITIKA_SONUC","SON5CEK_RENK_ORTALAMASI","SON5KULLANDIRIM","ŞUBE","SIRKET_TURU"]
for i in objects:  
    test[i] = le.fit_transform(test[i])

# Geleneksel ML Modeli

Direkt pickle ile modeli çekip test verisi sonuçlarına bakıyorum.

In [4]:
traditionalml = pickle.load(open('../Model Pickle/traditionalml.sav', 'rb'))
traditionalml

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.15, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=137,
              presort='auto', random_state=51, subsample=0.7, verbose=0,
              warm_start=False)

In [5]:
y_pred = traditionalml.predict(test)
cm = confusion_matrix(target, y_pred)
print("confusion_matrix:\n", cm)
print("accuracy_score: ", accuracy_score(target, y_pred))

confusion_matrix:
 [[1110   62]
 [ 182  221]]
accuracy_score:  0.8450793650793651


# Olasılık Bazlı ML Modeli
Olasılık bazlı modeli direkt çekerek gerekli fonksiyonları tanımlayarak test verisinde denemesini yapıyorum

In [6]:
olasilikbazli = pickle.load(open('../Model Pickle/olasilikbazlimodel.sav', 'rb'))
olasilikbazli

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.15, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=51, subsample=0.7, verbose=0,
              warm_start=False)

In [7]:
def preProcess(a):
    data=list(a.values())
    colz=list(a.keys())
    dfx=pd.DataFrame(data=[data], columns=colz)
    
    XX1 =np.array(dfx)
    XX2=dfx[features]
    XX = np.hstack((XX1))
    return XX


def cutoff(x):
    if x >= 0.50:
        return 1
    if x <= 0.50:
        return 0
    
def listdegerverme(liste):
    probs = []
    for i in liste:
        a = {'CEK_TUTAR':test.CEK_TUTAR[i], 'VADE_GUN':test.VADE_GUN[i], 'SON5ISTIHBARAT_SONUC':test.SON5ISTIHBARAT_SONUC[i], 
             'SON5POLITIKA_SONUC':test.SON5POLITIKA_SONUC[i],'SON5CEK_RENK_ORTALAMASI':test.SON5CEK_RENK_ORTALAMASI[i], 
             'SON5KULLANDIRIM':test.SON5KULLANDIRIM[i], 'ŞUBE':test.ŞUBE[i], 'SIRKET_TURU':test.SIRKET_TURU[i],
             'MUSTERI_RISK_SEVIYESI':test.MUSTERI_RISK_SEVIYESI[i], 'TK_KURUMSAYISISON5':test.TK_KURUMSAYISISON5[i], 
             'TK_GECIKMEHESAPSON5':test.TK_GECIKMEHESAPSON5[i],
             'TK_GECIKMEBAKIYEORT':test.TK_GECIKMEBAKIYEORT[i], 'TK_NAKDILIMIT':test.TK_NAKDILIMIT[i],
             'TK_NAKDIRISK':test.TK_NAKDIRISK[i],'TK_GAYRINAKDILIMIT':test.TK_GAYRINAKDILIMIT[i],
             'BK_KURUMSAYISI':test.BK_KURUMSAYISI[i],'BK_LIMIT':test.BK_LIMIT[i],
             'BK_RISK':test.BK_RISK[i],'BK_GECIKMEHESAP':test.BK_GECIKMEHESAP[i],
             'BK_GECIKMEBAKIYE':test.BK_GECIKMEBAKIYE[i],'T2_NAKDILIMIT_TPL':test.T2_NAKDILIMIT_TPL[i],
             'T2_NAKDIRISK_TPL':test.T2_NAKDIRISK_TPL[i], 'T2_NAKDIRISK_KV':test.T2_NAKDIRISK_KV[i],
             'T2_GAYRINAKDILIMIT':test.T2_GAYRINAKDILIMIT[i],'T2_GAYRINAKDIRISK':test.T2_GAYRINAKDIRISK[i], 
             'T2_FKTRNG_LIMIT':test.T2_FKTRNG_LIMIT[i],'T2_FKTRNG_TPLRISK':test.T2_FKTRNG_TPLRISK[i],
             'T2_NAKDIRISK_OV':test.T2_NAKDIRISK_OV[i],'T2_NAKDIRISK_UV':test.T2_NAKDIRISK_UV[i],
             'T2_SORUNLUKOD_RISK':test.T2_SORUNLUKOD_RISK[i],'T3_NAKDILIMIT_TPL':test.T3_NAKDILIMIT_TPL[i],
             'T3_NAKDIRISK_TPL':test.T3_NAKDIRISK_TPL[i],'T3_NAKDIRISK_KV':test.T3_NAKDIRISK_KV[i],
             'T3_NAKDIRISK_OV':test.T3_NAKDIRISK_OV[i], 'T3_NAKDIRISK_UV':test.T3_NAKDIRISK_UV[i],
             'T3_GAYRINAKDILIMIT':test.T3_GAYRINAKDILIMIT[i],'T3_GAYRINAKDIRISK':test.T3_GAYRINAKDIRISK[i],
             'T3_FKTRNG_LIMIT':test.T3_FKTRNG_LIMIT[i],'T3_SORUNLUKOD_RISK':test.T3_SORUNLUKOD_RISK[i],
             'T3_FKTRNG_TPLRISK':test.T3_FKTRNG_TPLRISK[i], 'T4_NAKDILIMIT_TPL':test.T4_NAKDILIMIT_TPL[i],
             'T4_NAKDIRISK_TPL':test.T4_NAKDIRISK_TPL[i],'T4_NAKDIRISK_KV':test.T4_NAKDIRISK_KV[i],
             'T4_NAKDIRISK_OV':test.T4_NAKDIRISK_OV[i],'T4_NAKDIRISK_UV':test.T4_NAKDIRISK_UV[i],
             'T4_GAYRINAKDILIMIT':test.T4_GAYRINAKDILIMIT[i],'T4_GAYRINAKDIRISK':test.T4_GAYRINAKDIRISK[i],
             'T4_FKTRNG_LIMIT':test.T4_FKTRNG_LIMIT[i],'T4_SORUNLUKOD_RISK':test.T4_SORUNLUKOD_RISK[i],
             'T4_FKTRNG_TPLRISK':test.T4_FKTRNG_TPLRISK[i]}
        XX=preProcess(a)
        XX = XX.reshape(1,-1)
        probs.append(olasilikbazli.predict_proba(XX)[:,1][0])
    liste = pd.DataFrame(liste)
    probs = pd.DataFrame(probs)
    testler = pd.concat([probs,liste],axis=1)
    testler.columns = ["Probs","İndex"]
    testler = testler.set_index("İndex")
    testler["WFunc"] = None
    testler["WFunc"] = testler.Probs.apply(cutoff)
    return np.array(testler.WFunc)

In [8]:
X_testindex = list(test.index)
probs = []
for i in X_testindex:
    a = {'CEK_TUTAR':test.CEK_TUTAR[i], 'VADE_GUN':test.VADE_GUN[i], 'SON5ISTIHBARAT_SONUC':test.SON5ISTIHBARAT_SONUC[i], 
         'SON5POLITIKA_SONUC':test.SON5POLITIKA_SONUC[i],'SON5CEK_RENK_ORTALAMASI':test.SON5CEK_RENK_ORTALAMASI[i], 
         'SON5KULLANDIRIM':test.SON5KULLANDIRIM[i], 'ŞUBE':test.ŞUBE[i], 'SIRKET_TURU':test.SIRKET_TURU[i],
         'MUSTERI_RISK_SEVIYESI':test.MUSTERI_RISK_SEVIYESI[i], 'TK_KURUMSAYISISON5':test.TK_KURUMSAYISISON5[i], 
         'TK_GECIKMEHESAPSON5':test.TK_GECIKMEHESAPSON5[i],
         'TK_GECIKMEBAKIYEORT':test.TK_GECIKMEBAKIYEORT[i], 'TK_NAKDILIMIT':test.TK_NAKDILIMIT[i],
         'TK_NAKDIRISK':test.TK_NAKDIRISK[i],'TK_GAYRINAKDILIMIT':test.TK_GAYRINAKDILIMIT[i],
         'BK_KURUMSAYISI':test.BK_KURUMSAYISI[i],'BK_LIMIT':test.BK_LIMIT[i],
         'BK_RISK':test.BK_RISK[i],'BK_GECIKMEHESAP':test.BK_GECIKMEHESAP[i],
         'BK_GECIKMEBAKIYE':test.BK_GECIKMEBAKIYE[i],'T2_NAKDILIMIT_TPL':test.T2_NAKDILIMIT_TPL[i],
         'T2_NAKDIRISK_TPL':test.T2_NAKDIRISK_TPL[i], 'T2_NAKDIRISK_KV':test.T2_NAKDIRISK_KV[i],
         'T2_GAYRINAKDILIMIT':test.T2_GAYRINAKDILIMIT[i],'T2_GAYRINAKDIRISK':test.T2_GAYRINAKDIRISK[i], 
         'T2_FKTRNG_LIMIT':test.T2_FKTRNG_LIMIT[i],'T2_FKTRNG_TPLRISK':test.T2_FKTRNG_TPLRISK[i],
         'T2_NAKDIRISK_OV':test.T2_NAKDIRISK_OV[i],'T2_NAKDIRISK_UV':test.T2_NAKDIRISK_UV[i],
         'T2_SORUNLUKOD_RISK':test.T2_SORUNLUKOD_RISK[i],'T3_NAKDILIMIT_TPL':test.T3_NAKDILIMIT_TPL[i],
         'T3_NAKDIRISK_TPL':test.T3_NAKDIRISK_TPL[i],'T3_NAKDIRISK_KV':test.T3_NAKDIRISK_KV[i],
         'T3_NAKDIRISK_OV':test.T3_NAKDIRISK_OV[i], 'T3_NAKDIRISK_UV':test.T3_NAKDIRISK_UV[i],
         'T3_GAYRINAKDILIMIT':test.T3_GAYRINAKDILIMIT[i],'T3_GAYRINAKDIRISK':test.T3_GAYRINAKDIRISK[i],
         'T3_FKTRNG_LIMIT':test.T3_FKTRNG_LIMIT[i],'T3_SORUNLUKOD_RISK':test.T3_SORUNLUKOD_RISK[i],
         'T3_FKTRNG_TPLRISK':test.T3_FKTRNG_TPLRISK[i], 'T4_NAKDILIMIT_TPL':test.T4_NAKDILIMIT_TPL[i],
         'T4_NAKDIRISK_TPL':test.T4_NAKDIRISK_TPL[i],'T4_NAKDIRISK_KV':test.T4_NAKDIRISK_KV[i],
         'T4_NAKDIRISK_OV':test.T4_NAKDIRISK_OV[i],'T4_NAKDIRISK_UV':test.T4_NAKDIRISK_UV[i],
         'T4_GAYRINAKDILIMIT':test.T4_GAYRINAKDILIMIT[i],'T4_GAYRINAKDIRISK':test.T4_GAYRINAKDIRISK[i],
         'T4_FKTRNG_LIMIT':test.T4_FKTRNG_LIMIT[i],'T4_SORUNLUKOD_RISK':test.T4_SORUNLUKOD_RISK[i],
         'T4_FKTRNG_TPLRISK':test.T4_FKTRNG_TPLRISK[i]}
    XX=preProcess(a)
    XX = XX.reshape(1,-1)
    probs.append(olasilikbazli.predict_proba(XX)[:,1][0])

X_testindex = pd.DataFrame(X_testindex)
probs = pd.DataFrame(probs)
testler = pd.concat([probs,X_testindex],axis=1)
testler.columns = ["Probs","İndex"]
testler = testler.set_index("İndex")
testler["WFunc"] = None
testler["WFunc"] = testler.Probs.apply(cutoff)

cm = confusion_matrix(target, testler.WFunc)
print("confusion_matrix:\n", cm)
print("accuracy_score: ", accuracy_score(target, testler.WFunc))

confusion_matrix:
 [[1093   79]
 [ 220  183]]
accuracy_score:  0.8101587301587302


# Keras Yaklaşımı
Keras modelinin içeriye import ederek değerlendirmesini yapıyorum

In [9]:
with open('../Model Pickle/model_architecture.json', 'r') as f:
    kerasmodel = model_from_json(f.read())

kerasmodel.load_weights('../Model Pickle/model_weights.h5')

In [10]:
def KerasPreProcess(test):
    scaler = MinMaxScaler((-1,1))
    return scaler.fit_transform(test)

def scalers(x):
    if x > 0.5:
        return 1
    if x <= 0.5:
        return 0

In [11]:
test2 = KerasPreProcess(test)

In [12]:
y_pred = kerasmodel.predict(test2)
y_pred = (y_pred > 0.5) * 1

cm = confusion_matrix(target, y_pred)
print("confusion_matrix:\n", cm)
print("accuracy_score: ", accuracy_score(target, y_pred))

confusion_matrix:
 [[1078   94]
 [ 219  184]]
accuracy_score:  0.8012698412698412


# Majority Vote Algoritması
Burada ise majority vote algoritması kuruluyor. Şu şekilde işliyor

- Bütün modeller verilen test verisi için sonuçları çıkartıyor.
- Bütün sonuçlar bir dataframede yan yana geliyor, ve Oylar sayılıp, kişinin churn edip etmeyeceği belirleniyor
- Bunun Görselleştirilmiş daha rahat hali için Sunum'un **14. Slaytına** bakabilirsiniz.

In [13]:
def MajorityVote(test):
    olasiliksonuc = listdegerverme(list(test.index))
    gelenekselml = traditionalml.predict(test)
    kerassonuc = kerasmodel.predict(KerasPreProcess(test))
    for i in range(0,len(kerassonuc)):
        kerassonuc[i] = scalers(kerassonuc[i])
    kerassonuc = pd.DataFrame(kerassonuc)
    gelenekselml = pd.DataFrame(gelenekselml)
    index = pd.DataFrame(list(test.index))
    gelenekselml = pd.concat([gelenekselml,index], axis=1)
    gelenekselml.columns = ["Sonuc","İndex"]
    gelenekselml.set_index("İndex")
    olasiliksonuc = pd.DataFrame(olasiliksonuc)
    gelenekselml = gelenekselml.Sonuc
    majority = pd.concat([olasiliksonuc,gelenekselml,kerassonuc],axis=1)
    majority["Majority"] = None
    majority.columns = ["Olasilik","Geleneksel","Keras","Majority"]
    for i in range(0,len(majority)):
        if majority.Olasilik[i] + majority.Geleneksel[i] + majority.Keras[i] == 3:
            majority.Majority[i] = 1
        elif majority.Olasilik[i] + majority.Geleneksel[i] + majority.Keras[i] == 2:
             majority.Majority[i] = 1
        elif majority.Olasilik[i] + majority.Geleneksel[i] + majority.Keras[i]== 1:
             majority.Majority[i] = 0
        elif majority.Olasilik[i] + majority.Geleneksel[i] + majority.Keras[i]== 0:
            majority.Majority[i] = 0
    return majority.Majority

In [14]:
Majority = MajorityVote(test)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http

- Bu kısımda ise çıktısını aldığımız majority vote sonucunu gerçek test sonuçları ile karşılaştırarak model'in doğruluğuna bakıyoruz.

In [15]:
Majority = pd.DataFrame(Majority)
index = pd.DataFrame(list(test.index))
Majority = pd.concat([Majority,index],axis=1)
Majority.columns = ["Majority","İndex"]
Majority = Majority.set_index("İndex")
Majority = Majority.astype(int)

In [16]:
cm = confusion_matrix(target, Majority)
print("confusion_matrix:\n", cm)
print("accuracy_score: ", accuracy_score(target, Majority))

confusion_matrix:
 [[1098   74]
 [ 193  210]]
accuracy_score:  0.8304761904761905
